In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [5]:
glove_wiki_vectors.most_similar('rain') # SOLUTION

[('rains', 0.8024376630783081),
 ('snow', 0.752005934715271),
 ('winds', 0.749399721622467),
 ('downpour', 0.7369566559791565),
 ('fog', 0.7337402105331421),
 ('wind', 0.7243762016296387),
 ('torrential', 0.7123619914054871),
 ('weather', 0.7077540159225464),
 ('storm', 0.6957893371582031),
 ('storms', 0.6879158616065979)]

In [6]:
glove_wiki_vectors.most_similar('meditation') # SOLUTION

[('yoga', 0.7812221646308899),
 ('transcendental', 0.6932344436645508),
 ('mindfulness', 0.692194402217865),
 ('meditative', 0.6846435070037842),
 ('contemplation', 0.6764698028564453),
 ('contemplative', 0.6538777351379395),
 ('spiritual', 0.6215183734893799),
 ('vipassana', 0.6162617206573486),
 ('buddhist', 0.6084271669387817),
 ('taoist', 0.6045626401901245)]

In [7]:
glove_wiki_vectors.most_similar('pomegranate') # SOLUTION

[('apricot', 0.7174665331840515),
 ('mango', 0.7000153660774231),
 ('tamarind', 0.6881566047668457),
 ('pumpkin', 0.6772226691246033),
 ('pear', 0.6767832040786743),
 ('pineapple', 0.6580570340156555),
 ('papaya', 0.6526557803153992),
 ('cranberry', 0.6495745182037354),
 ('peach', 0.6447383165359497),
 ('watermelon', 0.6376970410346985)]

In [8]:
glove_wiki_vectors.most_similar('ipad') # SOLUTION

[('iphone', 0.9220295548439026),
 ('ipod', 0.831965446472168),
 ('smartphone', 0.7812002301216125),
 ('android', 0.7511346936225891),
 ('app', 0.7433167099952698),
 ('ios', 0.7431976795196533),
 ('smartphones', 0.7359461784362793),
 ('apps', 0.7259055972099304),
 ('iphones', 0.7074244022369385),
 ('blackberry', 0.6987718343734741)]

In [9]:
glove_wiki_vectors.most_similar('cobain') # SOLUTION

[('nirvana', 0.7144757509231567),
 ('zeppelin', 0.5535445213317871),
 ('grunge', 0.5420762896537781),
 ('grohl', 0.5335221886634827),
 ('kurt', 0.5284802913665771),
 ('hendrix', 0.5253633260726929),
 ('jimi', 0.5184423923492432),
 ('nevermind', 0.5024071335792542),
 ('frontman', 0.4990401864051819),
 ('mellencamp', 0.49823328852653503)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [11]:
coast_shore_similarity = None
clothes_closet_similarity = None
old_new_similarity = None
smart_intelligent_similarity = None
dog_cat_similarity = None
tree_lawyer_similarity = None

# BEGIN SOLUTION
coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")
clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")

for (w1, w2) in word_pairs:
    print(
        "Cosine similarity between %s and %s = %0.3f"
        % (w1, w2, glove_wiki_vectors.similarity(w1, w2)))
    
# END SOLUTION   

Cosine similarity between coast and shore = 0.700
Cosine similarity between clothes and closet = 0.546
Cosine similarity between old and new = 0.643
Cosine similarity between smart and intelligent = 0.755
Cosine similarity between dog and cat = 0.880
Cosine similarity between tree and lawyer = 0.077


In [ ]:
grader.check("q1.2")

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [15]:
glove_wiki_vectors.similarity("white", "rich")

0.4472359

In [16]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [17]:
analogy("man", "intelligent", "woman") # SOLUTION

man : intelligent :: woman : ?


,Analogy word,Score
0,articulate,0.627857
1,thoughtful,0.616820
2,smart,0.587221
3,strong-willed,0.568862
4,vivacious,0.550627
5,opinionated,0.547024
6,perceptive,0.545705
7,inquisitive,0.544539
8,energetic,0.533960
9,literate,0.533894


In [18]:
glove_wiki_vectors.similarity("woman", "ugly"), glove_wiki_vectors.similarity("man", "ugly") # SOLUTION

(0.40388718, 0.44594428)

In [19]:
glove_wiki_vectors.similarity("man", "gentle"), glove_wiki_vectors.similarity("woman", "gentle") # SOLUTION

(0.40618372, 0.34959397)

In [20]:
glove_wiki_vectors.similarity("indigenous", "smart"), glove_wiki_vectors.similarity("white", "smart") # SOLUTION

(0.05271154, 0.32884747)

- The situation seems better than what we observed with the GoogleNews pre-trained embeddings. This is because most of the modern methods to compute word embeddings are debiased. 

- That said, we still see some stereotypes such as "white" it much more closer to "smart" than "indigenous" to "smart". That said, the similarity scores also depend upon the frequency of words, and if a certain word does not occur very frequently in the corpus, the similarity scores are going to be low for that word. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

1. The overall quality of embeddings looks good. It's able to find similar words for different types of words such as seasons, gadget names, artist names, and food items. The similarity scores make sense. The similarity between _coast_ and _shore_ is on the lower side. This might be the case because these words are not frequently occurring words. Note the high similarity score between _old_ and _new_. When we think of similarity, we usually think of synonyms. But the model is not able to distinguish between synonyms vs. antonyms. When words occur in similar contexts, they are similar according to the model.    

2. Perhaps NLP-based resume screening with such a model could lead to gender bias, say if it was used for hiring a computer programmer, which the model associates with men.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

**Classification with bag-of-words representation**

In [66]:
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

# BEGIN SOLUTION
from sklearn.metrics import classification_report

pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000))
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
print("Data matrix shape:", X_train_transformed.shape)
pipe.fit(X_train, y_train);

bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)
print("Train accuracy {:.2f}".format(bow_train_accuracy))
print("Test accuracy {:.2f}".format(bow_test_accuracy))
bow_report = classification_report(y_test, pipe.predict(X_test), digits=2)
print(bow_report)

# END SOLUTION

Data matrix shape: (9887, 8060)
Train accuracy 0.96
Test accuracy 0.82
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



**Classification with average embedding representation**

In [26]:
X_train_embeddings = None
X_test_embeddings = None

# BEGIN SOLUTION
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])
# END SOLUTION

We have reduced dimensionality from 13,064 to 300! 

In [27]:
X_train_embeddings.shape # SOLUTION

(9887, 300)

In [28]:
X_train_embeddings.head() # SOLUTION

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.690874,-0.755216,-3.279650,-1.111956,2.333812,1.712701,0.346985,5.052109,-1.078272,1.564646,...,0.978959,0.228331,0.843013,-1.340969,-0.884684,1.971317,-0.880458,-0.599687,-5.299727,-0.139515
1,0.021659,-1.051312,-4.063361,-1.818072,3.027572,-1.226410,0.035204,2.806751,-0.181556,1.530708,...,0.873182,-1.440154,0.219029,-1.426304,-1.591761,-0.475234,-0.122053,0.695197,-4.421231,-0.027261
2,-0.101639,-1.674547,-1.391433,-3.230565,0.636887,0.756545,0.205278,5.351284,-2.872737,3.033967,...,0.594443,0.435623,-0.556652,-2.759545,-1.521208,0.728254,-0.365033,1.230975,-5.989079,0.142662
3,-1.502830,-0.245505,-2.919757,-1.312146,3.315014,-0.281209,1.019417,3.909962,-0.440415,0.797455,...,0.452126,-0.261931,-0.375499,-0.218783,-1.355882,0.384430,0.619531,0.548577,-3.217188,0.119259
4,-0.503838,-0.450076,-4.626111,-2.820389,4.793096,-0.805586,1.518527,5.712772,1.548753,1.589522,...,-0.293538,-0.217686,-3.700689,-1.188137,0.648456,-2.548641,-0.413536,1.096762,-3.927443,-1.008204


In [67]:
emb_train_accuracy = None
emb_test_accuracy = None
emb_report = None # use digits=2

# BEGIN SOLUTION

lgr = LogisticRegression(max_iter=2000)
lgr.fit(X_train_embeddings, y_train)

emb_train_accuracy = lgr.score(X_train_embeddings, y_train)
emb_test_accuracy = lgr.score(X_test_embeddings, y_test)
print("Train accuracy {:.2f}".format(emb_train_accuracy))
print("Test accuracy {:.2f}".format(emb_test_accuracy))
emb_report = classification_report(y_test, lgr.predict(X_test_embeddings), digits=2)
print(emb_report)

# END SOLUTION

Train accuracy 0.85
Test accuracy 0.79
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.53       469
        exercise       0.67      0.73      0.70        74
         leisure       0.71      0.65      0.68       407
          nature       0.65      0.72      0.68        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



/Users/lzung/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
grader.check("q1.5")

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

Discussion: Logistic regression with bag of words would be more interpretable as there will be a coefficient associated with each word. The features of average embedding representation do not have a clear human interpretation. Transfer learning is not giving us any gains here. It might be worth trying out pre-trained twitter embeddings rather than Wikipedia embeddings on this dataset. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [33]:
from sklearn.datasets import fetch_20newsgroups

In [34]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [35]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [36]:
# BEGIN SOLUTION NO PROMPT
# nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])
nlp = en_core_web_md.load(disable=["parser", "ner"])
# END SOLUTION

In [37]:
# BEGIN SOLUTION NO PROMPT
def preprocess_spacy(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    text : (str)
        text to be preprocessed
    min_token_len : (int)
        minimum token length required to keep a token in the preprocessed text
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """
    clean_text = []
    
    for token in doc:
        if token.like_email:  # Check if the token is an not like email
            clean_text.append("EMAIL")
        elif token.like_url:  # Check if the token is an not like email
            clean_text.append("URL")
        elif token.like_num:  # Check if the token is an not like email
            clean_text.append("NUM")
        elif (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            clean_text.append(token.lemma_.lower())
    return " ".join(clean_text)

# END SOLUTION

In [38]:
# BEGIN SOLUTION NO PROMPT
import re
def preprocess(text):
    # Replace a sequence of whitespaces by a single whitespace
    text = re.sub(r"\s+", " ", text)
    
    # Remove other strange characters
    text = re.sub(r"""[\n\r]+""", "", text)
    
    # Remove other strange characters
    text = re.sub(r"""[\*\~]+""", "", text)

    # Replace slashes with spaces
    text = re.sub(r"""[\/]+""", " ", text)

    return text
# END SOLUTION

In [39]:
# BEGIN SOLUTION NO PROMPT
df["text"] = df["text"].apply(preprocess)
df.dropna()
# END SOLUTION

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,It's not a bad question: I don't have any ref...,1,comp.graphics
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics
3,"The following problem is really bugging me, an...",2,comp.windows.x
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone :: I am looking for some software ...,1,comp.graphics
4559,Archive-name: x-faq part3 Last-modified: 1993 ...,2,comp.windows.x
4560,"That's nice, but it doesn't answer the questi...",6,talk.politics.guns
4561,"Hi, I just got myself a Gateway 4DX-33V and tr...",2,comp.windows.x


In [40]:
# BEGIN SOLUTION NO PROMPT
df["text_pp"] = [
    preprocess_spacy(spacy_text) for spacy_text in nlp.pipe(df["text"].tolist())
] 
# END SOLUTION

Let's examine how does your preprocessed text look like for some examples.

In [41]:
# BEGIN SOLUTION NO PROMPT
df.iloc[:3]
# END SOLUTION

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read NUM u.s.c. NUM sence wonder help u.s...
1,It's not a bad question: I don't have any ref...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

_Type your answer here, replacing this text._

I decided to carry out the following preprocessing steps

- getting rid of slashes and new-line characters etc.
- sentence segmentation and tokenization      
- replaced all urls with token "URL" emails with token "EMAIL" and numbers with token "NUM" because these tokens in their original form are going to be quite unique. 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluded stopwords and punctuation because they are ubiquitous and unlikely to help us in topic modeling.  
- lemmatization   

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [42]:
# BEGIN SOLUTION NO PROMPT
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(df["text_pp"])
X_vec = vec.transform(df["text_pp"])
# END SOLUTION

In [43]:
# BEGIN SOLUTION NO PROMPT
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=123
)
document_topics = lda.fit_transform(X_vec)
# END SOLUTION

I picked 4 topics because in the categories we have chosen from this dataset, it seems like there could be four broad categories: religion, computers, sports, politics. Also, in my trial and error, I got the most sensible results with 4 topics on this dataset. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [44]:
# BEGIN SOLUTION NO PROMPT

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

# END SOLUTION

In [45]:
# BEGIN SOLUTION NO PROMPT
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=20,
)
# END SOLUTION

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           num           
game          people        god           url           
team          know          jesus         file          
year          think         entry         email         
play          say           church        use           
player        gun           bible         image         
good          time          christ        program       
win           come          file          window        
season        right         output        available     
go            thing         say           run           
get           state         faith         server        
think         go            know          include       
hockey        want          believe       widget        
league        believe       write         work          
run           tell          sin           display       
time          way           com

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [46]:
# BEGIN SOLUTION NO PROMPT
for i in range(5):
    print('\n\nDOCUMENT', df.iloc[i]['text'])
    print("\n\nDOCUMENT TOPIC ASSIGNMENT: ", document_topics[i]) 
    print('-----------')
# END SOLUTION    



DOCUMENT You know, I was reading 18 U.S.C. 922 and something just did not make sence and I was wondering if someone could help me out. Say U.S.C. 922 : (1) Except as provided in paragraph (2), it shall be unlawful for any person to transfer or possess a machinegun. Well I got to looking in my law dictionary and I found that a "person" might also be an artificial entity that is created by government and has no rights under the federal constitution. So, what I don't understand is how a statute like 922 can be enforced on an individual. So someone tell me how my government can tell me what I can or cannot possess. Just passing a law does not make it LAW. Everyone knows that laws are constitional until it goes to court. So, has it ever gone to court, not just your run of the mill "Ok I had it I am guilty, put me in jail" Has anyone ever claimed that they had a right to possess and was told by the Supreme Court that they didn't have that right? 


DOCUMENT TOPIC ASSIGNMENT:  [0.05744164 0

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

1. Here is how each model might be tackling multiclass classification. 
    - Decision trees: Natural extension to multi-class. The formula to calculate Gini index or information gain work for multiclass. 
    - KNNs: Natural extension to multiclass. We can just take a vote among multiple classes. 
    - RBF SVMs: We haven't talked about it yet but RBF SVMs use a kernel trick, which corresponds to transforming the original features into a new feature space and fitting a boundary there. But it's still dividing the boundary in half so we need multiclass meta strategy here. The default strategy used in `sklearn` is one-vs.-one. See [here](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification).   
    - Logistic regression: Above, it's using one-vs.-rest approach to handle multi-class. In this strategy a binary model is learned for each class that tries to separate that class from all of the other classes, resulting in as many binary models as there are classes. Note that there is also a multinomial logistic regression also called as **the maxent classifier**, which is different than the ovr meta strategy we are using above.
2. In the context of NLP and vision, training models from scratch is time consuming and resource intensive. A common practice is to download powerful pre-trained models which capture rich information about language and images and tuning these models for our specific tasks. This is called transfer learning. 
3. Because we could keep most of the transformations, and just replace the last one with something tailored to our task. 

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)